In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import transformers
import torch
import os
import nltk
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from jinja2 import Template
import pickle
from collections import defaultdict
import copy

from transformers.generation.logits_process import LogitsProcessorList
logits_processor = LogitsProcessorList()

import sys
sys.path.append('/scratch/users/bozyurt20/hpc_run/utilities')
sys.path.append("/scratch/users/bozyurt20/hpc_run/blobs/")
from util_research import *

max_len = 512
num_layers = 24
d_model = 4096

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0pp", truncation_side="right", add_prefix_space=True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0pp", device_map="balanced")

In [3]:
cosine_similarity = nn.CosineSimilarity(dim=0, eps=1e-6)

# Different Prompts

In [14]:
names_1 = ["John", "Harry", "Andrew", "Lisa", "Mary"]
names_2 = ["Henry", "David", "Sophia", "Olivia", "Emma"]

cities_1 = ["London", "Paris", "Oslo", "Istanbul", "Beijing"]
cities_2 = ["Sydney", "Cairo", "Seoul", "Rome", "Prague"]

class DataSample_TwoSentences():
    def __init__(self, name_1, city_1, city_2):
        self.name_1 = name_1
        self.city_1 = city_1
        self.city_2 = city_2
        self.context = name_1 + ", who is friends with the man who is from " + city_2 + ", travelled to " + city_1 + "."
        self.question_1 = "Where did " + name_1 + " travel to?"
        self.prompt_1 = self.context + " " + self.question_1
        self.answer_1 = city_1
        self.answer_2 = city_2
    def add_context_encoding(self, context_encoding):
        self.context_encoding = context_encoding
        
class SimResult():
    def __init__(self, context, answer, similarity_dict):
        self.context = context
        self.answer = answer
        self.similarity_dict = similarity_dict
        
def find_index(input_ids, entity_str):
    
    entity_id = tokenizer.encode(entity_str)
    
    if len(entity_id) != 2:
        print("Not an appropriate entity!")
        return
    
    entity_id = entity_id[0]
    
    input_ids_list = input_ids.tolist()

    all_entity_mention_indices = []
    for i, j in enumerate(input_ids_list[0]):
        if j == entity_id:
            all_entity_mention_indices.append(i)
    try:
        entity_ind = all_entity_mention_indices[0]
        return entity_ind
    except:
        print("entity not found in the input!")
        return
        
data_points_two_sentences = []
for name_1 in names_1 + names_2:
    for city_1 in cities_1:
        for city_2 in cities_2:
            sample = DataSample_TwoSentences(name_1, city_1, city_2)
            data_points_two_sentences.append(sample)
            sample = DataSample_TwoSentences(name_1, city_2, city_1)
            data_points_two_sentences.append(sample)


In [15]:
len(data_points_two_sentences)

500

In [13]:
for point in data_points_two_sentences:
    print(point.context)

John, who is friends with the man who is from Sydney, travelled to London.
John, who is friends with the man who is from London, travelled to Sydney.
John, who is friends with the man who is from Cairo, travelled to London.
John, who is friends with the man who is from London, travelled to Cairo.
John, who is friends with the man who is from Seoul, travelled to London.
John, who is friends with the man who is from London, travelled to Seoul.
John, who is friends with the man who is from Rome, travelled to London.
John, who is friends with the man who is from London, travelled to Rome.
John, who is friends with the man who is from Prague, travelled to London.
John, who is friends with the man who is from London, travelled to Prague.
John, who is friends with the man who is from Sydney, travelled to Paris.
John, who is friends with the man who is from Paris, travelled to Sydney.
John, who is friends with the man who is from Cairo, travelled to Paris.
John, who is friends with the man who

In [19]:
name_to_city = []

name_to_city_diff = []
name_to_city_diff_zero = []

for i, data_point in enumerate(data_points_two_sentences):
    
    context = data_point.context
    name_1 = data_point.name_1
    answer_1 = data_point.answer_1
    answer_2 = data_point.answer_2
    
    input_ids = tokenizer.encode(context, return_tensors="pt").to(model.encoder.device)
    
    with torch.no_grad():
        encoder_out = model.encoder(input_ids)
    
    name_1_index = find_index(input_ids, name_1)
    
    name_1_vector = encoder_out.last_hidden_state[0, name_1_index, :]
    
    answer_1_index = find_index(input_ids, answer_1)
    answer_2_index = find_index(input_ids, answer_2)
        
    answer_1_vector = encoder_out.last_hidden_state[0, answer_1_index, :]
    answer_2_vector = encoder_out.last_hidden_state[0, answer_2_index, :]
    
    sim11 = cosine_similarity(name_1_vector, answer_1_vector)
    sim12 = cosine_similarity(name_1_vector, answer_2_vector)
    
    if sim11 > sim12:
        name_to_city.append(1)
        name_to_city_diff.append(sim11 - sim12)
    else:
        name_to_city.append(0)
        name_to_city_diff_zero.append(sim12 - sim11)

new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code worki

In [20]:
print(sum(name_to_city)/len(name_to_city))
print(sum(name_to_city))
print(len(name_to_city))

0.412
206
500


In [24]:
print(sum(name_to_city_diff)/len(name_to_city_diff))
print(sum(name_to_city_diff))
print(len(name_to_city_diff))
print()
print(sum(name_to_city_diff_zero)/len(name_to_city_diff_zero))
print(sum(name_to_city_diff_zero))
print(len(name_to_city_diff_zero))
print()

tensor(0.0484, device='cuda:2')
tensor(9.9687, device='cuda:2')
206

tensor(0.0563, device='cuda:2')
tensor(16.5592, device='cuda:2')
294



In [21]:
answer_1_index

16

In [22]:
answer_2_index

11

In [23]:
name_1_index

0

# Regular Prompts

In [50]:
names_1 = ["John", "Harry", "Andrew", "Lisa", "Mary"]
names_2 = ["Henry", "David", "Sophia", "Olivia", "Emma"]

cities_1 = ["London", "Paris", "Oslo", "Istanbul", "Beijing"]
cities_2 = ["Sydney", "Cairo", "Seoul", "Rome", "Prague"]

class DataSample_TwoSentences():
    def __init__(self, name_1, name_2, city_1, city_2):
        self.name_1 = name_1
        self.name_2 = name_2
        self.city_1 = city_1
        self.city_2 = city_2
        self.context = name_1 + " travelled to " + city_1 + ". " + name_2 + " travelled to " + city_2 + "."
        self.question_1 = "Where did " + name_1 + " travel to?"
        self.question_2 = "Where did " + name_2 + " travel to?"
        self.prompt_1 = self.context + " " + self.question_1
        self.prompt_2 = self.context + " " + self.question_2
        self.answer_1 = city_1
        self.answer_2 = city_2
    def add_context_encoding(self, context_encoding):
        self.context_encoding = context_encoding
        
class SimResult():
    def __init__(self, context, answer, similarity_dict):
        self.context = context
        self.answer = answer
        self.similarity_dict = similarity_dict
        
def find_index(input_ids, entity_str):
    
    entity_id = tokenizer.encode(entity_str)
    
    if len(entity_id) != 2:
        print("Not an appropriate entity!")
        return
    
    entity_id = entity_id[0]
    
    input_ids_list = input_ids.tolist()

    all_entity_mention_indices = []
    for i, j in enumerate(input_ids_list[0]):
        if j == entity_id:
            all_entity_mention_indices.append(i)
    try:
        entity_ind = all_entity_mention_indices[0]
        return entity_ind
    except:
        print("entity not found in the input!")
        return
        
data_points_two_sentences = []
for name_1 in names_1:
    for name_2 in names_2:
        for city_1 in cities_1:
            for city_2 in cities_2:
                sample = DataSample_TwoSentences(name_1, name_2, city_1, city_2)
                data_points_two_sentences.append(sample)
    

In [5]:
len(data_points_two_sentences)

625

In [6]:
data_points_two_sentences[2].context

'John travelled to London. Henry travelled to Seoul.'

In [24]:
results = []

city_to_name = []
name_to_city = []

city_to_name_diff = []
city_to_name_diff_zero = []
name_to_city_diff = []
name_to_city_diff_zero = []

for i, data_point in enumerate(data_points_two_sentences):
    
    context = data_point.context
    name_1 = data_point.name_1
    name_2 = data_point.name_2
    answer_1 = data_point.answer_1
    answer_2 = data_point.answer_2
    
    input_ids = tokenizer.encode(context, return_tensors="pt").to(model.encoder.device)
    
    with torch.no_grad():
        encoder_out = model.encoder(input_ids)
    
    name_1_index = find_index(input_ids, name_1)
    name_2_index = find_index(input_ids, name_2)
    
    name_1_vector = encoder_out.last_hidden_state[0, name_1_index, :]
    name_2_vector = encoder_out.last_hidden_state[0, name_2_index, :]
    
    answer_1_index = find_index(input_ids, answer_1)
    answer_2_index = find_index(input_ids, answer_2)
        
    answer_1_vector = encoder_out.last_hidden_state[0, answer_1_index, :]
    answer_2_vector = encoder_out.last_hidden_state[0, answer_2_index, :]
    
    sim11 = cosine_similarity(name_1_vector, answer_1_vector)
    sim12 = cosine_similarity(name_1_vector, answer_2_vector)
    sim21 = cosine_similarity(name_2_vector, answer_1_vector)
    sim22 = cosine_similarity(name_2_vector, answer_2_vector)
    
    if sim11 > sim21:
        city_to_name.append(1)
        city_to_name_diff.append(sim11 - sim21)
    else:
        city_to_name.append(0)
        city_to_name_diff_zero.append(sim21 - sim11)
    if sim22 > sim12:
        city_to_name.append(1)
        city_to_name_diff.append(sim11 - sim21)
    else:
        city_to_name.append(0)
        city_to_name_diff_zero.append(sim21 - sim11)
        
    if sim11 > sim12:
        name_to_city.append(1)
        name_to_city_diff.append(sim11 - sim12)
    else:
        name_to_city.append(0)  
        name_to_city_diff_zero.append(sim12 - sim11)
        
    if sim22 > sim21:
        name_to_city.append(1)
        name_to_city_diff.append(sim22 - sim21)
    else:
        name_to_city.append(0)
        name_to_city_diff_zero.append(sim21 - sim22)

new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code worki

In [25]:
print(sum(name_to_city)/len(name_to_city))
print(sum(name_to_city))
print(len(name_to_city))
print()
print(sum(city_to_name)/len(city_to_name))
print(sum(city_to_name))
print(len(city_to_name))

0.9208
1151
1250

0.9264
1158
1250


In [26]:
print(sum(city_to_name_diff)/len(city_to_name_diff))
print(sum(city_to_name_diff))
print(len(city_to_name_diff))
print()
print(sum(city_to_name_diff_zero)/len(city_to_name_diff_zero))
print(sum(city_to_name_diff_zero))
print(len(city_to_name_diff_zero))
print()
print(sum(name_to_city_diff)/len(name_to_city_diff))
print(sum(name_to_city_diff))
print(len(name_to_city_diff))
print()
print(sum(name_to_city_diff_zero)/len(name_to_city_diff_zero))
print(sum(name_to_city_diff_zero))
print(len(name_to_city_diff_zero))
print()

tensor(0.0575, device='cuda:0')
tensor(66.6116, device='cuda:0')
1158

tensor(0.0289, device='cuda:0')
tensor(2.6553, device='cuda:0')
92

tensor(0.0814, device='cuda:0')
tensor(93.6862, device='cuda:0')
1151

tensor(0.0221, device='cuda:0')
tensor(2.1866, device='cuda:0')
99



In [47]:
max_similar_first_name = defaultdict(int)
max_similar_second_name = defaultdict(int)
max_similar_first_city = defaultdict(int)
max_similar_second_city = defaultdict(int)
second_max_similar_first_name = defaultdict(int)
second_max_similar_second_name = defaultdict(int)
second_max_similar_first_city = defaultdict(int)
second_max_similar_second_city = defaultdict(int)
third_max_similar_first_name = defaultdict(int)
third_max_similar_second_name = defaultdict(int)
third_max_similar_first_city = defaultdict(int)
third_max_similar_second_city = defaultdict(int)
fourth_max_similar_first_name = defaultdict(int)
fourth_max_similar_second_name = defaultdict(int)
fourth_max_similar_first_city = defaultdict(int)
fourth_max_similar_second_city = defaultdict(int)
least_max_similar_first_name = defaultdict(int)
least_max_similar_second_name = defaultdict(int)
least_max_similar_first_city = defaultdict(int)
least_max_similar_second_city = defaultdict(int)
second_least_max_similar_first_name = defaultdict(int)
second_least_max_similar_second_name = defaultdict(int)
second_least_max_similar_first_city = defaultdict(int)
second_least_max_similar_second_city = defaultdict(int)

for i, data_point in enumerate(data_points_two_sentences):
    
    context = data_point.context
    name_1 = data_point.name_1
    name_2 = data_point.name_2
    answer_1 = data_point.answer_1
    answer_2 = data_point.answer_2
    
    input_ids = tokenizer.encode(context, return_tensors="pt").to(model.encoder.device)
    
    with torch.no_grad():
        encoder_out = model.encoder(input_ids)
    
    name_1_index = find_index(input_ids, name_1)
    name_2_index = find_index(input_ids, name_2)
    
    name_1_vector = encoder_out.last_hidden_state[0, name_1_index, :]
    name_2_vector = encoder_out.last_hidden_state[0, name_2_index, :]
    
    answer_1_index = find_index(input_ids, answer_1)
    answer_2_index = find_index(input_ids, answer_2)
        
    answer_1_vector = encoder_out.last_hidden_state[0, answer_1_index, :]
    answer_2_vector = encoder_out.last_hidden_state[0, answer_2_index, :]
    
    similarity_dict = []
    for j in range(len(input_ids[0])):
        paired_token_vector = encoder_out.last_hidden_state[0, j, :]
        sim = cosine_similarity(paired_token_vector, answer_1_vector)
        similarity_dict.append((j, sim))
    similarity_dict.sort(key=lambda x: x[1], reverse=True)
    
    sorting = [ tok_id for tok_id, _  in similarity_dict]
    
    max_similar_first_city[sorting[1]] += 1
    second_max_similar_first_city[sorting[2]] += 1
    third_max_similar_first_city[sorting[3]] += 1
    fourth_max_similar_first_city[sorting[4]] += 1
    least_max_similar_first_city[sorting[-1]] += 1
    second_least_max_similar_first_city[sorting[-2]] += 1
    
    similarity_dict = []
    for j in range(len(input_ids[0])):
        paired_token_vector = encoder_out.last_hidden_state[0, j, :]
        sim = cosine_similarity(paired_token_vector, answer_2_vector)
        similarity_dict.append((j, sim))
    similarity_dict.sort(key=lambda x: x[1], reverse=True)
    
    sorting = [ tok_id for tok_id, _  in similarity_dict]
    
    max_similar_second_city[sorting[1]] += 1
    second_max_similar_second_city[sorting[2]] += 1
    third_max_similar_second_city[sorting[3]] += 1
    fourth_max_similar_second_city[sorting[4]] += 1
    least_max_similar_second_city[sorting[-1]] += 1
    second_least_max_similar_second_city[sorting[-2]] += 1
    
    similarity_dict = []
    for j in range(len(input_ids[0])):
        paired_token_vector = encoder_out.last_hidden_state[0, j, :]
        sim = cosine_similarity(paired_token_vector, name_1_vector)
        similarity_dict.append((j, sim))
    similarity_dict.sort(key=lambda x: x[1], reverse=True)
    
    sorting = [ tok_id for tok_id, _  in similarity_dict]
    
    max_similar_first_name[sorting[1]] += 1
    second_max_similar_first_name[sorting[2]] += 1
    third_max_similar_first_name[sorting[3]] += 1
    fourth_max_similar_first_name[sorting[4]] += 1
    least_max_similar_first_name[sorting[-1]] += 1
    second_least_max_similar_first_name[sorting[-2]] += 1
    
    similarity_dict = []
    for j in range(len(input_ids[0])):
        paired_token_vector = encoder_out.last_hidden_state[0, j, :]
        sim = cosine_similarity(paired_token_vector, name_2_vector)
        similarity_dict.append((j, sim))
    similarity_dict.sort(key=lambda x: x[1], reverse=True)
    
    sorting = [ tok_id for tok_id, _  in similarity_dict]
    
    max_similar_second_name[sorting[1]] += 1
    second_max_similar_second_name[sorting[2]] += 1
    third_max_similar_second_name[sorting[3]] += 1
    fourth_max_similar_second_name[sorting[4]] += 1
    least_max_similar_second_name[sorting[-1]] += 1
    second_least_max_similar_second_name[sorting[-2]] += 1
    

new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code worki

In [39]:
sorting

[6, 0, 5, 10, 7, 8, 11, 1, 9, 2, 3, 4, 12]

In [40]:
print(max_similar_first_city)
print(max_similar_second_city)
print(max_similar_first_name)
print(max_similar_second_name)

defaultdict(<class 'int'>, {3: 345, 10: 280})
defaultdict(<class 'int'>, {9: 387, 4: 237, 3: 1})
defaultdict(<class 'int'>, {6: 625})
defaultdict(<class 'int'>, {0: 574, 5: 51})


In [41]:
print(second_max_similar_first_city)
print(second_max_similar_second_city)
print(second_max_similar_first_name)
print(second_max_similar_second_name)

defaultdict(<class 'int'>, {10: 345, 3: 280})
defaultdict(<class 'int'>, {4: 388, 9: 184, 3: 53})
defaultdict(<class 'int'>, {1: 160, 5: 427, 4: 38})
defaultdict(<class 'int'>, {5: 574, 0: 51})


In [43]:
print(third_max_similar_first_city)
print(third_max_similar_second_city)
print(third_max_similar_first_name)
print(third_max_similar_second_name)

defaultdict(<class 'int'>, {9: 578, 0: 32, 6: 7, 2: 8})
defaultdict(<class 'int'>, {3: 500, 6: 102, 9: 14, 8: 9})
defaultdict(<class 'int'>, {5: 148, 4: 75, 1: 402})
defaultdict(<class 'int'>, {11: 71, 1: 27, 10: 401, 4: 49, 8: 34, 7: 36, 9: 7})


In [45]:
print(fourth_max_similar_first_city)
print(fourth_max_similar_second_city)
print(fourth_max_similar_first_name)
print(fourth_max_similar_second_name)

defaultdict(<class 'int'>, {0: 484, 2: 49, 6: 83, 9: 7, 8: 2})
defaultdict(<class 'int'>, {6: 456, 3: 71, 8: 91, 0: 2, 11: 5})
defaultdict(<class 'int'>, {4: 273, 5: 48, 3: 31, 2: 194, 10: 10, 11: 3, 1: 63, 7: 3})
defaultdict(<class 'int'>, {10: 83, 1: 103, 11: 78, 9: 93, 8: 167, 4: 13, 7: 88})


In [48]:
print(least_max_similar_first_city)
print(least_max_similar_second_city)
print(least_max_similar_first_name)
print(least_max_similar_second_name)

defaultdict(<class 'int'>, {12: 599, 9: 26})
defaultdict(<class 'int'>, {12: 604, 9: 21})
defaultdict(<class 'int'>, {12: 592, 9: 33})
defaultdict(<class 'int'>, {12: 593, 9: 32})


In [49]:
print(second_least_max_similar_first_city)
print(second_least_max_similar_second_city)
print(second_least_max_similar_first_name)
print(second_least_max_similar_second_name)

defaultdict(<class 'int'>, {7: 584, 12: 26, 11: 4, 9: 6, 8: 5})
defaultdict(<class 'int'>, {1: 521, 2: 35, 12: 21, 9: 11, 7: 13, 0: 24})
defaultdict(<class 'int'>, {8: 86, 10: 77, 12: 33, 9: 429})
defaultdict(<class 'int'>, {3: 521, 12: 32, 9: 8, 4: 62, 2: 1, 7: 1})


In [51]:
for i, data_point in enumerate(data_points_two_sentences):
    
    context = data_point.context
    input_ids = tokenizer.encode(context, return_tensors="pt").to(model.encoder.device)    
    with torch.no_grad():
        encoder_out = model.encoder(input_ids)        
    data_point.add_context_encoding(encoder_out.last_hidden_state)


new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code working-modeling_t5
new code worki

In [66]:
max_similar_first_to = defaultdict(int)
max_similar_second_to = defaultdict(int)
second_max_similar_first_to = defaultdict(int)
second_max_similar_second_to = defaultdict(int)
third_max_similar_first_to = defaultdict(int)
third_max_similar_second_to = defaultdict(int)
fourth_max_similar_first_to = defaultdict(int)
fourth_max_similar_second_to = defaultdict(int)
least_similar_first_to = defaultdict(int)
least_similar_second_to = defaultdict(int)
second_least_similar_first_to = defaultdict(int)
second_least_similar_second_to = defaultdict(int)

for i, data_point in enumerate(data_points_two_sentences):
    
    context = data_point.context
    context_encoding = data_point.context_encoding
    
    input_ids = tokenizer.encode(context, return_tensors="pt").to(model.encoder.device)
    
    to_1_index = 3
    to_2_index = 9
    
    to_1_vector = context_encoding[0, to_1_index, :]
    to_2_vector = context_encoding[0, to_2_index, :]
    
    similarity_dict = []
    for j in range(len(input_ids[0])):
        paired_token_vector = context_encoding[0, j, :]
        sim = cosine_similarity(paired_token_vector, to_1_vector)
        similarity_dict.append((j, sim))
    similarity_dict.sort(key=lambda x: x[1], reverse=True)
    
    sorting = [ tok_id for tok_id, _  in similarity_dict]
    max_similar_first_to[sorting[1]] += 1
    second_max_similar_first_to[sorting[2]] += 1
    third_max_similar_first_to[sorting[3]] += 1
    fourth_max_similar_first_to[sorting[4]] += 1
    least_similar_first_to[sorting[-1]] += 1
    second_least_similar_first_to[sorting[-2]] += 1
    
    similarity_dict = []
    for j in range(len(input_ids[0])):
        paired_token_vector = context_encoding[0, j, :]
        sim = cosine_similarity(paired_token_vector, to_2_vector)
        similarity_dict.append((j, sim))
    similarity_dict.sort(key=lambda x: x[1], reverse=True)
    
    sorting = [ tok_id for tok_id, _  in similarity_dict]
    max_similar_second_to[sorting[1]] += 1
    second_max_similar_second_to[sorting[2]] += 1
    third_max_similar_second_to[sorting[3]] += 1
    fourth_max_similar_second_to[sorting[4]] += 1
    least_similar_second_to[sorting[-1]] += 1
    second_least_similar_second_to[sorting[-2]] += 1

In [67]:
print(max_similar_first_to)
print(max_similar_second_to)
print(second_max_similar_first_to)
print(second_max_similar_second_to)
print(third_max_similar_first_to)
print(third_max_similar_second_to)
print(fourth_max_similar_first_to)
print(fourth_max_similar_second_to)

defaultdict(<class 'int'>, {9: 585, 2: 40})
defaultdict(<class 'int'>, {3: 585, 12: 40})
defaultdict(<class 'int'>, {2: 585, 4: 39, 8: 1})
defaultdict(<class 'int'>, {10: 114, 8: 249, 7: 36, 2: 222, 3: 4})
defaultdict(<class 'int'>, {4: 568, 8: 53, 10: 4})
defaultdict(<class 'int'>, {8: 278, 10: 235, 2: 69, 11: 40, 7: 2, 4: 1})
defaultdict(<class 'int'>, {8: 568, 10: 39, 4: 18})
defaultdict(<class 'int'>, {2: 283, 8: 56, 1: 30, 10: 234, 11: 3, 4: 11, 3: 6, 7: 2})


In [68]:
print(least_similar_first_to)
print(least_similar_second_to)
print(second_least_similar_first_to)
print(second_least_similar_second_to)

defaultdict(<class 'int'>, {12: 617, 9: 8})
defaultdict(<class 'int'>, {12: 579, 8: 27, 0: 8, 6: 11})
defaultdict(<class 'int'>, {7: 530, 12: 8, 9: 23, 11: 26, 6: 38})
defaultdict(<class 'int'>, {7: 112, 0: 472, 4: 3, 6: 30, 1: 1, 8: 5, 12: 1, 5: 1})


In [74]:
a = torch.rand(3, 4)

In [75]:
b = torch.zeros_like(a)

In [76]:
b

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [77]:
torch.zeros(5)

tensor([0., 0., 0., 0., 0.])